In [10]:
!pip uninstall sagemaker -y
!pip uninstall sagemaker -y
!pip install --no-cache-dir sagemaker

Found existing installation: sagemaker 3.0
Uninstalling sagemaker-3.0:
  Successfully uninstalled sagemaker-3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 129.1 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of sagemaker-serve to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of sagemaker-serve to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.


In [3]:
import boto3

sts = boto3.client("sts")
account_id = sts.get_caller_identity()["Account"]

print("Account:", account_id)

Account: 619071324395


In [2]:
!pip list | grep sagemaker

sagemaker                 3.0
sagemaker-core            2.4.1
sagemaker_pyspark         1.4.5


In [7]:
!pip uninstall -y sagemaker sagemaker-core sagemaker_pyspark
!pip install sagemaker==2.221.0

Found existing installation: sagemaker 3.0
Uninstalling sagemaker-3.0:
  Successfully uninstalled sagemaker-3.0
Found existing installation: sagemaker-core 2.4.1
Uninstalling sagemaker-core-2.4.1:
  Successfully uninstalled sagemaker-core-2.4.1
Found existing installation: sagemaker_pyspark 1.4.5
Uninstalling sagemaker_pyspark-1.4.5:
  Successfully uninstalled sagemaker_pyspark-1.4.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.0 MB/s  0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1
  Attempting uninstall: cloudpickle━━━━━━━━━━━━━ 0/4 [protobuf]
    Found existing installation: cloudpickle 3.1.20/4 [protobuf]
    Uninstalling cloudpickle-3.1.2:━━━━━━━━━ 0/4 [protobuf]
      Successfully uninstalled cloudpickle-3.1.2 0/4 [protobuf]
  Attempting uninstall: attrs━━━━━━━━━━━━━━━ 0/4 [protobuf]
    Found existing installation: attrs 25.4.0 0/4 [protobuf]
  

In [1]:
from sagemaker import get_execution_role
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
print("Environment ready ✅")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Environment ready ✅


In [2]:
hyperparameters = {
    "model_id": "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T",
    "epochs": 2,
    "per_device_train_batch_size": 2,
    "lr": 2e-5
}

In [10]:
instance_type="ml.m5.xlarge"

In [11]:
estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./scripts",
    role=role,
    transformers_version="4.36",
    pytorch_version="2.1",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    output_path="s3://llm-model-artifacts-bucket/models/",
    hyperparameters=hyperparameters
)

In [12]:
estimator.fit({
    "train": "s3://llm-finetune-datset-bucket/datasets/"
})



INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2026-02-22-06-32-25-378


ResourceLimitExceeded: An error occurred (ResourceLimitExceeded) when calling the CreateTrainingJob operation: The account-level service limit 'ml.g5.xlarge for training job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.

In [ ]:
estimator.model_data

In [ ]:
# Use this code to check all the accessible services inside your AWS Sagemaker

from sagemaker import image_uris

image_uris.retrieve(
    framework="huggingface",
    region="ap-south-1",   # change your region
    version="4.37.0",
    image_scope="inference"
)

In [ ]:
## the code for the deployment
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

role = sagemaker.get_execution_role()

model = HuggingFaceModel(
    model_data="s3://llm-model-artifacts-bucket/models/huggingface-pytorch-training-2025-11-15-11-42-14-536/output/model.tar.gz",
    role=role,
    transformers_version="4.37.0",
    pytorch_version="2.1.0",
    py_version="py310",
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    endpoint_name="live-finetune-endpoint"
)